# **Requirements**

Python 3.8.10

pythonnet 2.5.2

`pip install pythonnet==2.5.2`

In [ ]:
from base_interactive import PythonZOSConnection
from beam_shaper import BeamShaper
import sys
from importlib import metadata

print("Python version: ", sys.version)
print('Pythonnet version:', metadata.version('pythonnet'))

Python version:  3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
Pythonnet version: 2.5.2


# **Connect to Zemax and initiate a system**

In [ ]:
zos = PythonZOSConnection()
ZOSAPI = zos.ZOSAPI
TheApplication = zos.TheApplication
TheSystem = zos.TheSystem

# TheSystem.Mode = ZOSAPI.ZOSAPI_Mode.Plugin
TheSystem.UpdateMode = ZOSAPI.LensUpdateMode.AllWindows

print('Connected to OpticStudio')
# The connection should now be ready to use.  For example:
print('Serial #: ', TheApplication.SerialCode)

print('ZOSAPI Mode: ', TheSystem.Mode)
print("Lens update mode: ", TheSystem.UpdateMode)

Connected to OpticStudio
Serial #:  20120530
ZOSAPI Mode:  0
Lens update mode:  2


# **Set System Data**

In [278]:
# Define System Explore
SysExplore = TheSystem.SystemData

## **Set Title and Notes**

In [279]:
# Set Title and Notes
SysExplore.TitleNotes.Title = "Paraxial Zoom Lens Generator"
SysExplore.TitleNotes.Notes = "Generate a paraxial zoom lens based on the calculation of the positive and negative compensated zoom lens formulas."
SysExplore.TitleNotes.Author = "Ziyi Xiong"

## **Set Aperture**

In [280]:
# Set Aperture
SysExplore.Aperture.ApertureType = ZOSAPI.SystemData.ZemaxApertureType.ImageSpaceFNum
SysExplore.Aperture.ApertureValue = 5.6

## **Set Fields**

In [281]:
# Set Fields
SysExplore.Fields.SetFieldType(ZOSAPI.SystemData.FieldType.ParaxialImageHeight)
SysExplore.Fields.ApplyFieldWizard(ZOSAPI.SystemData.FieldPattern.EqualAreaY, 9, 6.6, 0, 0, 0, True, False)

## **Set Wavelengths**

### **Via Presets**

In [282]:
# SysExplore.Wavelengths.SelectWavelengthPreset(ZOSAPI.SystemData.WavelengthPreset.FdC_Visible)

### **Customize Wavelengths**

In [283]:
# Set Wavelengths Values of Cellphone lens
num_wavelengths = SysExplore.Wavelengths.NumberOfWavelengths

if num_wavelengths == 1:
    SysExplore.Wavelengths.GetWavelength(1).Wavelength = 0.5876
    SysExplore.Wavelengths.GetWavelength(1).Weight = 24
    SysExplore.Wavelengths.AddWavelength(0.6563, 11)
    SysExplore.Wavelengths.AddWavelength(0.5461, 24)
    SysExplore.Wavelengths.AddWavelength(0.4861, 12)
    SysExplore.Wavelengths.AddWavelength(0.4360, 3)
    SysExplore.Wavelengths.AddWavelength(0.4047, 1)
    
num_wavelengths = SysExplore.Wavelengths.NumberOfWavelengths
    
print("Number of wavelengths after insertion: ", num_wavelengths)

Number of wavelengths after insertion:  6


### **Remove Wavelengths**

In [284]:
# if num_wavelengths > 1: [SysExplore.Wavelengths.RemoveWavelength(i) for i in range(num_wavelengths, 1, -1)]

# **Set Lens Data**

In [285]:
SysLDE = TheSystem.LDE

## **Add Surfaces**

In [286]:
num_surfaces = SysLDE.NumberOfSurfaces

if num_surfaces == 3:
    for i in range(4): # range(4) = [0, 1, 2, 3]
        SysLDE.AddSurface()

num_surfaces = SysLDE.NumberOfSurfaces
print("Number of surfaces after insertion: ", num_surfaces)

Number of surfaces after insertion:  7


## **Get and Set Surface Type**

In [287]:
# Use a list to store all surfaces objects
Surface=[SysLDE.GetSurfaceAt(i) for i in range(0, num_surfaces)] 

Paraxial_Surface = ZOSAPI.Editors.LDE.SurfaceType.Paraxial

# Change Surface Types to Paraxial
for i in range(2, num_surfaces - 1):
    st = SysLDE.GetSurfaceAt(i).GetSurfaceTypeSettings(Paraxial_Surface)
    Surface[i].ChangeType(st)

## **Set Multi-Configuration**

In [288]:
SysMCE = TheSystem.MCE

### **Add Configurations**

In [289]:
num_configs = SysMCE.NumberOfConfigurations

if num_configs == 1:
    SysMCE.AddConfiguration(False)
    
num_configs = SysMCE.NumberOfConfigurations

### **Add and Set Operands**

In [290]:
num_operands = SysMCE.NumberOfOperands

if num_operands == 1:
    for i in range(2):
        SysMCE.AddOperand()
        
num_operands = SysMCE.NumberOfOperands

print("Number of Operands after insertion: ", num_operands)

# Use a list to store all MC operands objects        
MC_Operand=[SysMCE.GetOperandAt(i) for i in range(1, num_operands + 1)]
MC_Operand.insert(0, None)  # to make the index start from 1

print(MC_Operand)
print(MC_Operand[1])

THIC = ZOSAPI.Editors.MCE.MultiConfigOperandType.THIC

# Set Surface Numbers of the Operands
for i in range(1, num_operands + 1):
    MC_Operand[i].ChangeType(THIC)
    MC_Operand[i].Param1 = i + 1  # Surface Number

Number of Operands after insertion:  3
[None, <ZemaxUI.ZOSAPI.Editors.ZOSAPI_MCERow object at 0x000001799260EC40>, <ZemaxUI.ZOSAPI.Editors.ZOSAPI_MCERow object at 0x000001799260EEE0>, <ZemaxUI.ZOSAPI.Editors.ZOSAPI_MCERow object at 0x000001799260E6A0>]
ZemaxUI.ZOSAPI.Editors.ZOSAPI_MCERow


## **Set Surface Data**

### **Set Stop**

In [291]:
Surface[3].IsStop = True 

### **Set Paraxial Focal Length**

In [292]:
Paraxial_Focal_Length = ZOSAPI.Editors.LDE.SurfaceColumn.Par1

# Get the values of the PCZL parameters
pczl = pczl.PCZL(f_3 = 1.2, m_4 = 3, d_12s = 0.5, d_34s = 0.5, q = 0.2785, num_samples=101)

Surface[2].SurfaceData.Par1.DoubleValue = pczl.f_1
Surface[3].SurfaceData.Par1.DoubleValue = pczl.f_2
Surface[4].SurfaceData.Par1.DoubleValue = pczl.f_3
Surface[5].SurfaceData.Par1.DoubleValue = pczl.f_4

### **Set Comments and Thickness**

In [ ]:
Surface[1].Comment = "Dummy"
Surface[2].Comment = "Front Fixed Group"
Surface[3].Comment = "Variator"
Surface[4].Comment = "Compensator"
Surface[5].Comment = "Rear Fixed Group"

Surface[1].Thickness = 1.0

# Thickness values for short focal length position
MC_Operand[1].GetOperandCell(1).DoubleValue = pczl.d_12s    # Thickness of Surface 2
MC_Operand[2].GetOperandCell(1).DoubleValue = pczl.d_23[-1] # Thickness of Surface 3
MC_Operand[3].GetOperandCell(1).DoubleValue = pczl.d_34s    # Thickness of Surface 4

# Thickness values for long focal length position
MC_Operand[1].GetOperandCell(2).DoubleValue = pczl.d_12[0]  # Thickness of Surface 2
MC_Operand[2].GetOperandCell(2).DoubleValue = pczl.d_23[0]  # Thickness of Surface 3
MC_Operand[3].GetOperandCell(2).DoubleValue = pczl.d_34[0]  # Thickness of Surface 4

Surface[5].Thickness = pczl.l_4i

# **Export Cross Section**

In [294]:
# Export Cross Section
SysLayout = TheSystem.Tools.Layouts.OpenCrossSectionExport()

print("Number of configurations: ", num_configs)

for i in range(1, num_configs + 1):
    SysLayout.StartSurface = -1
    # SysLayout.EndSurface = num_surfaces
    SysLayout.NumberOfRays = 5
    SysLayout.Wavelength = -1
    SysLayout.Field = -1
    # SysLayout.MarginalAndChiefRayOnly = True
    SysLayout.OutputPixelWidth = 1920
    SysLayout.OutputPixelHeight = 1080
    SysLayout.OutputFileName = fr"C:\Users\xiong\Desktop\CrossSection_Config{i}.png"
    SysLayout.SaveImageAsFile = True
    SysLayout.Configuration = i
    SysLayout.RunAndWaitForCompletion()

SysLayout.Close()

Number of configurations:  2


True